### Ingest pit_stops.json file

##### Step 1 - Read the Json file using the spark dataframe reader API - Multiline Json

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col, lit
from pyspark.sql.functions import to_timestamp, concat, col, current_timestamp, to_date

In [0]:
qualifying_schema = StructType(fields=[StructField("qualifyId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), True),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("constructorId", IntegerType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("position", StringType(), True),
                                      StructField("q1", StringType(), True),
                                      StructField("q2", StringType(), True),
                                      StructField("q3", StringType(), True),
                                      ])


In [0]:
qualifying_df = spark.read \
.schema(qualifying_schema) \
.option("multiLine", True) \
.json("/mnt/formula1stg/raw/qualifying")

In [0]:
#display(qualifying_df)

####Renaming columns

In [0]:
final_df = qualifying_df.withColumnRenamed("qualifyId", "qualify_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("constructorId", "constructor_id") \
.withColumn("data_source", lit(v_data_source)) \
.withColumn("ingestion_date", current_timestamp()) 



In [0]:
# final_df.write.mode("overwrite").parquet("/mnt/formula1stg/processed/qualifying")

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.qualifying")

In [0]:
display(spark.read.parquet("/mnt/formula1stg/processed/qualifying"))

In [0]:
dbutils.notebook.exit("Success")